In [1]:
import logging
import threading
import json
import itertools
#import modin.pandas as pd
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import ibm_db
import shap

from pandas_profiling import ProfileReport
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions.db import Database
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.enginelog import EngineLogging
from iotfunctions.dbtables import FileModelStore
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.dbtables import FileModelStore, DBModelStore
from mmfunctions.anomaly import (ARIMAForecaster, SpectralAnomalyScoreExt)
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
from sklearn import metrics
import scipy as sp
import scipy.fftpack
import skimage as ski  
from skimage import util as skiutil # for nifty windowing
import pyod as pyod
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

In [2]:
db_schema=None
class DatabaseDummy:
    tenant_id = '###_IBM_###'
    db_type = 'db2'
    model_store = FileModelStore('./')
    def _init(self):
        return

db = DatabaseDummy()
        
print (db.tenant_id)

2021-03-29T20:50:06.883 INFO iotfunctions.dbtables.__init__ Init FileModelStore with path: ./
###_IBM_###


In [3]:
# Get stuff in


df_input_raw = pd.read_csv('./BP_DATA/IoTData.csv', index_col=False, parse_dates=['DATETIME'], comment='#')

df_input_raw = df_input_raw.rename(columns={'DATETIME':'timestamp'}).\
                drop(columns=['UMIDADE_REL','KW','KW2'])
temp_air = sp.signal.resample(df_input_raw['TEMP_AIR'].astype(np.float64), 212840)

df_input_raw['entity'] = 'demand'
#df_input_raw['entity']=df_input_raw['DEVICE_ID']
#df_input_raw['timestamp']=df_input_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_input_raw = df_input_raw.sort_values(by='timestamp').dropna()
df_input_raw['timestamp'] = pd.to_datetime(df_input_raw['timestamp'])
df_input = df_input_raw.set_index(['timestamp']).resample("T", label="right").ffill()

#df_input

df_input


2021-03-29T20:50:06.953 INFO numexpr.utils._init_num_threads Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-03-29T20:50:06.954 INFO numexpr.utils._init_num_threads NumExpr defaulting to 8 threads.


,TEMP_AIR,entity
timestamp,,
2019-09-15 00:00:00,18.7,demand
2019-09-15 00:01:00,18.7,demand
2019-09-15 00:02:00,18.7,demand
2019-09-15 00:03:00,18.7,demand
2019-09-15 00:04:00,18.7,demand
...,...,...
2020-08-31 19:56:00,19.0,demand
2020-08-31 19:57:00,19.0,demand
2020-08-31 19:58:00,19.0,demand


In [4]:
df_input['TEMP_AIR'] = df_input['TEMP_AIR'].astype(np.float64)

In [5]:
df_input

,TEMP_AIR,entity
timestamp,,
2019-09-15 00:00:00,18.7,demand
2019-09-15 00:01:00,18.7,demand
2019-09-15 00:02:00,18.7,demand
2019-09-15 00:03:00,18.7,demand
2019-09-15 00:04:00,18.7,demand
...,...,...
2020-08-31 19:56:00,19.0,demand
2020-08-31 19:57:00,19.0,demand
2020-08-31 19:58:00,19.0,demand


In [6]:
ll = 5

df_large_list = []
#df_raw = df_input_raw.reset_index().set_index(['timestamp'])
for i in range(ll):
    df_new = df_input.copy()
    df_new['entity'] = 'entity' + str(i)
    df_large_list.append(df_new)

df_large = pd.concat(df_large_list)

In [7]:
df_large = df_large.reset_index().set_index(['entity','timestamp'])

In [8]:
df_large['TEMP_AIR'] = df_large['TEMP_AIR'].astype(np.float64)
df_large.index.levels[0].name

'entity'

In [9]:

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}


EngineLogging.configure_console_logging(logging.DEBUG)

spsi = SpectralAnomalyScoreExt('TEMP_AIR', 12, 'Spectral', 'SpectralInv')
et = spsi._build_entity_type(columns = [Column('TEMP_AIR',Float())], **jobsettings)
spsi._entity_type = et
df_large = spsi.execute(df = df_large)


2021-03-29T20:50:15.560 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2021-03-29T20:50:15.562 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-29T20:50:15.562 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-29T20:50:15.563 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 8.3.1
2021-03-29T20:50:15.564 DEBUG iotfunctions.util.__init__ Starting trace
2021-03-29T20:50:15.564 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_SpectralAnomalyScoreExt_20210329185015
2021-03-29T20:50:15.564 DEBUG iotfunctions.util.__init__ auto_save None
2021-03-29T20:50:15.565 DEBUG iotfunctions.util.categorize_args categorizing arguments
2021-03-29T20:50:15.565 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
2021-03-29T20:50:15.566 DEBUG iotfunctions.metadata.__init__ Initializ

2021-03-29T20:50:19.592 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-29T20:50:19.604 DEBUG mmfunctions.anomaly._calc ->
2021-03-29T20:50:20.030 DEBUG mmfunctions.anomaly._calc -->
2021-03-29T20:50:20.036 DEBUG mmfunctions.anomaly._calc --->
2021-03-29T20:50:21.674 DEBUG mmfunctions.anomaly.execute Spectral done


In [13]:
df_large.index

MultiIndex([('entity0', '2019-09-15 00:00:00'),
            ('entity0', '2019-09-15 00:01:00'),
            ('entity0', '2019-09-15 00:02:00'),
            ('entity0', '2019-09-15 00:03:00'),
            ('entity0', '2019-09-15 00:04:00'),
            ('entity0', '2019-09-15 00:05:00'),
            ('entity0', '2019-09-15 00:06:00'),
            ('entity0', '2019-09-15 00:07:00'),
            ('entity0', '2019-09-15 00:08:00'),
            ('entity0', '2019-09-15 00:09:00'),
            ...
            ('entity4', '2020-08-31 19:51:00'),
            ('entity4', '2020-08-31 19:52:00'),
            ('entity4', '2020-08-31 19:53:00'),
            ('entity4', '2020-08-31 19:54:00'),
            ('entity4', '2020-08-31 19:55:00'),
            ('entity4', '2020-08-31 19:56:00'),
            ('entity4', '2020-08-31 19:57:00'),
            ('entity4', '2020-08-31 19:58:00'),
            ('entity4', '2020-08-31 19:59:00'),
            ('entity4', '2020-08-31 20:00:00')],
           names=['enti

In [11]:
df_large.index.levels[0][0]

'entity0'